In [ ]:
import numpy as np
import mdtraj as md
import pickle

stride = 1
total_frames = 10000
total_time_ns = 100

timestep  = (stride/total_frames) * total_time_ns

nframes = int(np.ceil(total_frames/stride))


topfile='colpm_pyc_complex/sys_prod/start.pdb'
xtcfile='colpm_pyc_complex/sys_prod/prod_colpm_pyc_complex_compact_compact.xtc'

full_traj = md.load_xtc(xtcfile, stride=stride, top=topfile)
pyc_atoms = full_traj.topology.select('resname PYC')
hbonds = md.baker_hubbard(full_traj, periodic=False, freq=0.01)
label = lambda hbond : '%s - %s ... %s' % ( full_traj.topology.atom(hbond[0]),\
                                            full_traj.topology.atom(hbond[1]),\
                                            full_traj.topology.atom(hbond[2]))
pyc_hbonds = []
for hbond in hbonds:
    d,h,a = hbond
    if d in pyc_atoms or a in pyc_atoms:
        pyc_hbonds.append(hbond)
        print(label(hbond))

pyc_hbonds = np.array(pyc_hbonds)

# List of variables to pickle
variables_to_pickle = {
    'hbonds': hbonds,
    'nframes': nframes,
    'pyc_atoms': pyc_atoms,
    'pyc_hbonds': pyc_hbonds,
    'stride': stride,
    'timestep': timestep,
    'total_frames': total_frames,
    'total_time_ns': total_time_ns
}

# Dump each variable to a separate pickle file
for var_name, var_value in variables_to_pickle.items():
    with open(f'pickles/{var_name}.pkl', 'wb') as f:
        pickle.dump(var_value, f)